In [21]:
import psutil
import os
from collections import Counter
from typing import List, Tuple
import re
import ujson as json  # Usar ujson para un análisis JSON más rápido
from concurrent.futures import ThreadPoolExecutor

# Compilar la expresión regular una sola vez
pattern = re.compile(r'@(\w+)')

# Función para procesar un grupo de líneas y extraer nombres de usuario
def process_lines(lines: List[str]) -> Counter:
    usernames = Counter()
    for line in lines:
        try:
            data = json.loads(line.strip())
            if 'content' in data:
                content = data['content']
                # Extraer los nombres de usuario
                usernames.update(pattern.findall(content))
        except json.JSONDecodeError:
            continue
    return usernames

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    chunk_size = 30  # Tamaño del lote de líneas para procesar
    
    total_counter = Counter()
    
    # Usar ThreadPoolExecutor para paralelizar el procesamiento de líneas
    with ThreadPoolExecutor(max_workers=8) as executor:  # Ajustar basado en el número de núcleos de CPU
        with open(file_path, 'r') as file:
            # Leer y procesar el archivo en bloques
            chunk = []
            for line in file:
                chunk.append(line)
                if len(chunk) >= chunk_size:
                    future = executor.submit(process_lines, chunk)
                    total_counter.update(future.result())
                    chunk = []
            # Procesar el último bloque si hay líneas restantes
            if chunk:
                future = executor.submit(process_lines, chunk)
                total_counter.update(future.result())
    
    # Top 10 nombres de usuario
    top_10_usernames = total_counter.most_common(10)
    
    return top_10_usernames

# Ruta del archivo JSON a procesar
file_path = '/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4-large.json'
top_10_usernames = q3_time(file_path)
print(top_10_usernames)


[('narendramodi', 22610), ('Kisanektamorcha', 18360), ('RakeshTikaitBKU', 16390), ('PMOIndia', 14220), ('RahulGandhi', 11250), ('GretaThunberg', 10460), ('RaviSinghKA', 10150), ('rihanna', 9720), ('UNHumanRights', 9620), ('meenaharris', 9250)]


In [20]:
from collections import Counter
from typing import List, Tuple
import re
import ujson as json
from concurrent.futures import ThreadPoolExecutor
import mmap

# Compilar la expresión regular una sola vez
pattern = re.compile(r'@(\w+)')

def process_lines(lines: List[bytes]) -> Counter:
    local_counter = Counter()
    for line in lines:
        try:
            data = json.loads(line.decode().strip())
            if 'content' in data:
                content = data['content']
                local_counter.update(pattern.findall(content))
        except json.JSONDecodeError:
            continue
    return local_counter

def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    usernames = Counter()
    with open(file_path, 'r') as file:
        with mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Convertir el mmap en bytes y dividir por nuevas líneas
            lines = mm.read().splitlines()

    # Dividir las líneas en bloques y procesar en paralelo
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(process_lines, [lines[i:i+1000] for i in range(0, len(lines), 1000)]))
    
    # Combinar los contadores
    for result in results:
        usernames.update(result)
    
    # Top 10 nombres de usuario
    top_10_usernames = usernames.most_common(10)
    
    return top_10_usernames



Uso de memoria antes de cargar el archivo: 325.5625 MB
Uso de memoria después de procesar los datos: 240.78125 MB
Uso de memoria después de contar los nombres de usuario: 240.78125 MB


[('narendramodi', 22610),
 ('Kisanektamorcha', 18360),
 ('RakeshTikaitBKU', 16390),
 ('PMOIndia', 14220),
 ('RahulGandhi', 11250),
 ('GretaThunberg', 10460),
 ('RaviSinghKA', 10150),
 ('rihanna', 9720),
 ('UNHumanRights', 9620),
 ('meenaharris', 9250)]

In [19]:
from collections import Counter
from typing import List, Tuple
import re
import ujson as json

# Compilar la expresión regular una sola vez
pattern = re.compile(r'@(\w+)')

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    usernames = Counter()
    
    # Leer el archivo en bloques para mejorar el rendimiento
    chunk_size = 1024 * 1024  # 1 MB
    with open(file_path, 'r') as file:
        buffer = ''
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            buffer += chunk
            lines = buffer.split('\n')
            
            # Procesar todas las líneas menos la última, que puede estar incompleta
            for line in lines[:-1]:
                try:
                    data = json.loads(line.strip())
                    if 'content' in data:
                        content = data['content']
                        # Extraer los nombres de usuario
                        usernames.update(pattern.findall(content))
                except json.JSONDecodeError:
                    continue
            
            # Mantener la última línea parcial en el buffer
            buffer = lines[-1]
    
    # Procesar la última línea si queda algo en el buffer
    if buffer:
        try:
            data = json.loads(buffer.strip())
            if 'content' in data:
                content = data['content']
                usernames.update(pattern.findall(content))
        except json.JSONDecodeError:
            pass
    
    # Top 10 nombres de usuario
    top_10_usernames = usernames.most_common(10)
    
    return top_10_usernames

file_path = '/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4.json'
q3_time(file_path)

[('narendramodi', 2261),
 ('Kisanektamorcha', 1836),
 ('RakeshTikaitBKU', 1639),
 ('PMOIndia', 1422),
 ('RahulGandhi', 1125),
 ('GretaThunberg', 1046),
 ('RaviSinghKA', 1015),
 ('rihanna', 972),
 ('UNHumanRights', 962),
 ('meenaharris', 925)]

In [20]:
from collections import Counter
from typing import List, Tuple
import re
import ujson as json

# Compilar la expresión regular una sola vez
pattern = re.compile(r'@(\w+)')

def q3_time_optimized(file_path: str) -> List[Tuple[str, int]]:
    usernames = Counter()
    
    # Leer el archivo línea por línea para evitar cargar todo en memoria
    with open(file_path, 'r') as file:
        for line in file:
            try:
                data = json.loads(line.strip())
                if 'content' in data:
                    content = data['content']
                    # Extraer los nombres de usuario y actualizar el Counter
                    usernames.update(pattern.findall(content))
            except json.JSONDecodeError:
                continue
    
    # Top 10 nombres de usuario
    top_10_usernames = usernames.most_common(10)
    
    return top_10_usernames


file_path = '/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4.json'
q3_time(file_path)

[('narendramodi', 2261),
 ('Kisanektamorcha', 1836),
 ('RakeshTikaitBKU', 1639),
 ('PMOIndia', 1422),
 ('RahulGandhi', 1125),
 ('GretaThunberg', 1046),
 ('RaviSinghKA', 1015),
 ('rihanna', 972),
 ('UNHumanRights', 962),
 ('meenaharris', 925)]